# Winnowing 算法交互式测试（基于词级别的 k-grams）

本 Notebook 提供了一个基于**词级别**的 Winnowing 算法实现，用于快速测试和理解其基本原理。
Winnowing 是一种用于文档指纹提取的算法，常用于检测文档相似性，特别是抄袭检测。它通过选择文本中词级别 k-grams 哈希值的"最小值"作为文档的指纹，从而实现对局部内容变化的鲁棒性。

主要步骤包括：
1.  **文本预处理**：将文本转换为规范形式（例如，小写，标准化空格）。
2.  **分词**：对于英文，以空格分词；对于中文，使用jieba分词。
3.  **生成词级别 k-grams**：将文本分割成固定数量（k）的连续词语序列。
4.  **哈希 k-grams**：将每个词级别 k-gram 转换为一个整数哈希值。
5.  **Winnowing 选择**：在哈希值序列上使用一个滑动窗口（大小为 w），从每个窗口中选择一个哈希值（通常是最小值）作为指纹。为了确保选中所有匹配，如果窗口中有多个最小值，则选择最右边的那一个。
6.  **相似度计算**：使用选定的指纹集合，通过 Jaccard 相似度等方法比较文档。

您可以修改下面的代码单元中的样本文本和参数（`k_value` 和 `window_size`）来观察 Winnowing 算法的效果。相比于字符级别的 k-grams，词级别的 k-grams 通常更能捕捉文本的语义相似性。

In [ ]:
import re
import hashlib
import jieba  # 导入jieba用于中文分词

def preprocess_text(text: str) -> str:
    """
    预处理文本：转换为小写，标准化空格，但保留词的分隔符。
    返回预处理后的文本，准备用于分词。
    """
    text = text.lower()
    # 标准化空格但不完全移除标点符号，因为我们要在后面做词级分词
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def get_word_k_grams(text: str, k: int) -> list[str]:
    """
    从文本中提取词级别的 k-grams。
    对于中文，使用jieba分词。
    对于英文，使用空格分词。
    
    参数:
    text (str): 输入文本
    k (int): k-gram 中连续词的数量
    
    返回:
    list[str]: 词级别的 k-gram 列表，每个 k-gram 是由下划线连接的 k 个连续词
    """
    processed_text = preprocess_text(text)
    
    # 判断文本是否包含中文字符
    has_chinese = any('\u4e00' <= char <= '\u9fff' for char in processed_text)
    
    # 分词
    if has_chinese:
        # 中文文本使用jieba分词
        words = [word for word in jieba.cut(processed_text) if word.strip() and not re.fullmatch(r'[\W_]+', word)]
    else:
        # 英文文本先移除标点，然后按空格分词
        text_no_punct = re.sub(r'[^\w\s]', '', processed_text)
        words = [word for word in text_no_punct.split() if word.strip()]
    
    # 如果单词数量少于k，无法构成k-gram
    if len(words) < k:
        if words:  # 如果有词但不足k个，将它们合并为一个k-gram
            return ["_".join(words)]
        return []
    
    # 生成词级别的k-grams
    k_grams = []
    for i in range(len(words) - k + 1):
        k_gram = "_".join(words[i:i+k])
        k_grams.append(k_gram)
    
    return k_grams

def hash_k_gram(k_gram: str) -> int:
    """
    为 k-gram 生成一个哈希值。
    """
    # 为了演示，我们使用简单的hash。更可靠的做法是：
    # return int(hashlib.sha256(k_gram.encode('utf-8')).hexdigest(), 16) % (2**32)
    return hash(k_gram)

def winnowing(k_grams_hashes: list[int], w: int) -> list[int]:
    """
    应用 Winnowing 算法从哈希列表中选择指纹。
    w: 窗口大小，即考虑的连续哈希值的数量。
    """
    if not k_grams_hashes:
        return []
    if w <= 0: # 窗口大小必须为正
        # 如果窗口大小无效，返回独特的哈希值
        return sorted(list(set(k_grams_hashes)))

    fingerprints = {} # 使用字典 {hash_value: position} 来存储指纹

    # 遍历所有可能的窗口
    for i in range(len(k_grams_hashes) - w + 1):
        window = k_grams_hashes[i : i + w]
        
        # 在当前窗口中找到最小值及其位置
        min_hash_in_window = float('inf')
        min_hash_global_pos = -1
        
        for j in range(w - 1, -1, -1): # 从右向左遍历窗口
            current_hash_in_window = window[j]
            if current_hash_in_window <= min_hash_in_window: # 使用 <= 来确保选择最右边的最小值
                min_hash_in_window = current_hash_in_window
                min_hash_global_pos = i + j
        
        if min_hash_global_pos != -1:
            fingerprints[min_hash_in_window] = min_hash_global_pos

    return sorted(list(fingerprints.keys())) # 返回排序后的唯一指纹哈希值

def jaccard_similarity(set1: list[int], set2: list[int]) -> float:
    """计算两个指纹集合之间的 Jaccard 相似度。"""
    s1 = set(set1)
    s2 = set(set2)
    
    intersection = len(s1.intersection(s2))
    union = len(s1.union(s2))
    
    if union == 0: # 处理两个集合都为空的情况
        return 1.0 if intersection == 0 else 0.0 
    return intersection / union

print("Winnowing 核心函数已定义。")

Winnowing 核心函数已定义。


### 示例：比较两段文本

下面的单元格演示了如何使用上面定义的函数来比较两段文本。
您可以修改 `text1`, `text2`, `k_value` (k-gram 长度) 和 `window_size` (Winnowing 窗口大小) 来进行实验。

In [ ]:
# --- 参数设置 ---
k_value = 2  # k-gram 中词的数量（例如：2表示bigram）
window_size = 3 # Winnowing 算法的窗口大小 (w)

# --- 样本文本 ---
text1 = "The quick brown fox jumps over the lazy dog."
text2 = "A quick brown dog jumps over the lazy fox."
# 尝试一些中文文本
# text1 = "窗前明月光，疑是地上霜。"
# text2 = "床前明月光，疑是地上霜。举头望明月，低头思故乡。"

# 1. 生成词级别的 k-grams
word_k_grams1 = get_word_k_grams(text1, k_value)
word_k_grams2 = get_word_k_grams(text2, k_value)

print(f"文本1原始文本: '{text1}'")
print(f"文本1预处理: '{preprocess_text(text1)}'")
print(f"文本1的词级 {k_value}-grams: {word_k_grams1}")
print("-" * 50)

print(f"文本2原始文本: '{text2}'")
print(f"文本2预处理: '{preprocess_text(text2)}'")
print(f"文本2的词级 {k_value}-grams: {word_k_grams2}")
print("-" * 50)

# 2. 哈希 k-grams
hashes1 = [hash_k_gram(kg) for kg in word_k_grams1]
hashes2 = [hash_k_gram(kg) for kg in word_k_grams2]

print(f"文本1的哈希值: {hashes1}")
print(f"文本2的哈希值: {hashes2}")
print("-" * 50)

# 3. 应用 Winnowing 算法选择指纹
fingerprints1 = winnowing(hashes1, window_size)
fingerprints2 = winnowing(hashes2, window_size)

print(f"文本1的指纹 (共 {len(fingerprints1)} 个): {fingerprints1}")
print(f"文本2的指纹 (共 {len(fingerprints2)} 个): {fingerprints2}")
print("-" * 50)

# 4. 计算 Jaccard 相似度
similarity = jaccard_similarity(fingerprints1, fingerprints2)
print(f"词级 k-gram 大小 (k): {k_value}")
print(f"窗口大小 (w): {window_size}")
print(f"文本1和文本2之间的 Jaccard 相似度为: {similarity:.4f}")

common_fingerprints = set(fingerprints1).intersection(set(fingerprints2))
print(f"共同指纹数量: {len(common_fingerprints)}")

# 如果想查看共同的指纹对应哪些词级k-grams，可以反向查找
if len(common_fingerprints) > 0 and len(common_fingerprints) < 10:  # 只在共同指纹较少时显示详情
    print("\n共同指纹对应的词级 k-grams:")
    for fp_hash in common_fingerprints:
        # 在哈希列表中找到这个值的位置
        if fp_hash in hashes1:
            idx1 = hashes1.index(fp_hash)
            print(f"- 哈希值 {fp_hash}: '{word_k_grams1[idx1]}'")
        if fp_hash in hashes2:
            idx2 = hashes2.index(fp_hash)
            print(f"  对应于文本2中的: '{word_k_grams2[idx2]}'")

处理后的文本1: thequickbrownfoxjumpsoverthelazydog
处理后的文本2: aquickbrowndogjumpsoverthelazyfox
------------------------------
文本1的 5-grams (前10个): ['thequ', 'hequi', 'equic', 'quick', 'uickb', 'ickbr', 'ckbro', 'kbrow', 'brown', 'rownf']
文本2的 5-grams (前10个): ['aquic', 'quick', 'uickb', 'ickbr', 'ckbro', 'kbrow', 'brown', 'rownd', 'owndo', 'wndog']
------------------------------
文本1的哈希值 (前10个): [7508795006550652616, 4176976264316860555, 3368108420909165039, 1812050576779763094, 7147763509316105680, -8375326502675596680, -2041526832594093505, 5624359777173550237, 1798523942293157633, 2737555228708556518]
文本2的哈希值 (前10个): [-1216561883296411258, 1812050576779763094, 7147763509316105680, -8375326502675596680, -2041526832594093505, 5624359777173550237, 1798523942293157633, 8193173288735182965, -7885635568996073102, -3440476585849518327]
------------------------------
文本1的指纹 (共 11 个): [-8790618685562707809, -8514884714470023795, -8494937997708778057, -8375326502675596680, -7044782622179392102, -40801

### 尝试修改

现在您可以：
1.  修改上面单元格中的 `text1` 和 `text2`。
2.  调整 `k_value` 和 `window_size`。
    *   `k_value` 表示词级别 k-gram 中连续词的数量。
    *   较小的 `k` (如 1 或 2) 会产生更多通用的词组匹配，适合检测广泛相似性。
    *   较大的 `k` (如 3 或更多) 会更具特异性，适合检测更精确的短语匹配。
    *   `window_size` 影响指纹的密度。较小的 `w` 会产生更密集的指纹，较大的 `w` 会产生更稀疏的指纹。Winnowing 算法保证，如果两段文本有一个由连续 `t = k + w - 1` 个词组成的完全匹配的短语，那么它们至少会共享一个指纹。
3.  尝试中文文本（取消 `text1` 和 `text2` 的注释），看看基于词级别的 k-gram 如何处理中文文本。
4.  重新运行上面的 Python 单元格以查看结果如何变化。

**注意**：由于我们使用了词级别的 k-grams 而不是字符级别的 k-grams，`k_value` 的合理值通常比字符级别的小。例如，对于词级别，`k_value=2` 或 `k_value=3` 通常就足够了，因为这已经代表了 2-3 个连续词的序列。